In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np
import re
import pprint
from collections import OrderedDict

In [3]:
! head ftes_lorvec.out

#RPA-results:
#Nucleus: SN 146
#Excitation: 1 -
#Total number of pairs: 2188
#Number of ph-/ah-pairs: 1554/634
#Maximal excitation-energy for particle-hole-pairs: 200
#Maximal excitation-energy for antiparticle-hole-pairs: 2000
#2 imaginary eigenvalues
#Listing imaginary eigenvalues:
#Imaginary part: 4.383495e-01


In [4]:
146 - 50 # N

96

In [5]:
# 'energy', 'transition_strength'
df_lorvec = pd.read_csv('ftes_lorvec.out', delim_whitespace=True, comment='#', skip_blank_lines=True,
            header=None, names=['U', 'fE1'])

In [6]:
df_lorvec = df_lorvec[(df_lorvec.U >= 0.1) & (df_lorvec.U <= 30)] # MeV

In [7]:
df_lorvec.head()

,U,fE1
10,0.10,0.135900
11,0.11,0.136607
12,0.12,0.137321
13,0.13,0.138042
14,0.14,0.138770


In [8]:
every_10 = df_lorvec.iloc[::10, :]
every_10.reset_index(drop=True, inplace=True)
every_10_T = every_10.T

In [9]:
df = every_10_T
df2 = pd.concat([df], keys=[146], names=['A'])
df3 = pd.concat([df2], keys=[50], names=['Z'])
lorvec_df = df3

In [10]:
stocks = pd.read_csv('http://bit.ly/smallstocks')

In [11]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

In [12]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()

In [13]:
ser.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['Symbol', 'Date'])

In [14]:
ser.unstack()

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [15]:
df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')

In [16]:
stocks.set_index(['Symbol', 'Date'], inplace=True)

In [17]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

In [18]:
stocks.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[1, 0, 2, 0, 2, 1, 2, 1, 0], [0, 0, 0, 1, 1, 1, 2, 2, 2]],
           names=['Symbol', 'Date'])

In [19]:
stocks.sort_index(inplace=True)

In [20]:
def to_float(iterable):
    return (float(val) for val in iterable)

In [21]:
class ConfigurationSyntaxError(Exception):
    pass

In [24]:
# @TODO 
# df_to_dict(df):
    # get dictionary from dataframe X
# @TODO 
# dict_to_fn(ordered_nested_dict, fname):
    # write dict to file 
# @TODO same float formatting
# @TODO multiply by constant to convert e^2fm^2 to barn
# @TODO why missing some A values

In [22]:
def fn_to_dict(fname):
    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    with open(fname) as f:
        contents = f.read()
    
    new_line = re.compile('\n[\s\r]+?\n')
    blocks = new_line.split(contents)
    
    assert blocks[-1].strip() == '', "Last line not empty!"
    assert len(blocks[:-1]) == 82, "Not right number of blocks!"
    
    mydict = OrderedDict()
    mydict[z_from_fname] =  OrderedDict()
    
    minA, maxA = 400, 0
    for blk in blocks[:-1]: # except last line
        block = blk.splitlines()
    
        nucleus_header = block[0].split()
        Z, A = (int(val) for val in nucleus_header[1::2])
        if (nucleus_header[0::2] != ['Z=', 'A=']) or (Z != z_from_fname):
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)
        if int(A) < minA:
            minA = int(A)
        if int(A) > maxA:
            maxA = int(A)
        
        col_header = block[1].split()
        if col_header[:2] != ['U[MeV]', 'fE1[mb/MeV]']:
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)

        columns = (row.split() for row in block[2:])
        c1_vals, c2_vals = zip(*columns)

        d = OrderedDict()
        d[A] = OrderedDict(U=tuple(to_float(c1_vals)),
                         fE1=tuple(to_float(c2_vals)))
        mydict[Z].update(d)
        
    print(f"Amax - Amin = {maxA - minA}, {len(blocks[:-1])} blocks")
    return mydict, minA, maxA

In [89]:
def dict_to_fn(ordered_nested_dict, fname):
    ond = ordered_nested_dict
    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    print(z_from_fname)
#     print(ond[z_from_fname])
    nucleus_header = f' Z={z_from_fname:4d} A={90:4d}'
    print(nucleus_header)
    col_header = f'  U[MeV]  fE1[mb/MeV]'
    print(col_header)
    data = '{:9.3f}   {:.3E}'.format(11.100, 6.510E-03)
    print(data)
    
    for Z in ond.keys():
        for A in ond[Z].keys():
            print(Z, A)
            for U, fE1 in zip(ond[Z][A]['U'], ond[Z][A]['fE1']):
                print(U, fE1)
    
    with open(fname, "w") as f:
        f.write('This is not a test.\n')


In [86]:
t1 = (1,2,3)
t2 = (4,5,6)

for a, b in zip(t1, t2):
    print(a, b)

1 4
2 5
3 6


In [82]:
md, a, b = fn_to_dict('z050')
print(f"Amin = {a}, Amax = {b}")

Amax - Amin = 88, 82 blocks
Amin = 90, Amax = 178


In [90]:
dict_to_fn(md, 'newz050')

50
 Z=  50 A=  90
  U[MeV]  fE1[mb/MeV]
   11.100   6.510E-03
50 90
0.1 0.006291
0.2 0.006776
0.3 0.0073
0.4 0.007862
0.5 0.008465
0.6 0.009107
0.7 0.00979
0.8 0.01051
0.9 0.01128
1.0 0.01209
1.1 0.01294
1.2 0.01383
1.3 0.01478
1.4 0.01576
1.5 0.0168
1.6 0.01788
1.7 0.01902
1.8 0.02021
1.9 0.02145
2.0 0.02275
2.1 0.02412
2.2 0.02554
2.3 0.02703
2.4 0.02859
2.5 0.03023
2.6 0.03195
2.7 0.03376
2.8 0.03567
2.9 0.03769
3.0 0.03985
3.1 0.04216
3.2 0.04467
3.3 0.04745
3.4 0.0506
3.5 0.05432
3.6 0.05901
3.7 0.06546
3.8 0.07461
3.9 0.08496
4.0 0.09324
4.1 0.0997
4.2 0.1053
4.3 0.1105
4.4 0.1156
4.5 0.1206
4.6 0.1252
4.7 0.1296
4.8 0.1338
4.9 0.1382
5.0 0.1428
5.1 0.1477
5.2 0.153
5.3 0.1585
5.4 0.1644
5.5 0.1706
5.6 0.1772
5.7 0.1842
5.8 0.1916
5.9 0.1995
6.0 0.208
6.1 0.2173
6.2 0.2272
6.3 0.2377
6.4 0.2486
6.5 0.2594
6.6 0.2696
6.7 0.279
6.8 0.2877
6.9 0.2961
7.0 0.305
7.1 0.3146
7.2 0.3251
7.3 0.3367
7.4 0.3493
7.5 0.3631
7.6 0.378
7.7 0.3941
7.8 0.4115
7.9 0.4302
8.0 0.4502
8.1 0.4716
8.2 

20.3 6.467
20.4 6.284
20.5 6.101
20.6 5.918
20.7 5.737
20.8 5.558
20.9 5.382
21.0 5.211
21.1 5.044
21.2 4.882
21.3 4.726
21.4 4.575
21.5 4.43
21.6 4.292
21.7 4.159
21.8 4.032
21.9 3.91
22.0 3.794
22.1 3.684
22.2 3.578
22.3 3.476
22.4 3.379
22.5 3.286
22.6 3.197
22.7 3.111
22.8 3.028
22.9 2.949
23.0 2.872
23.1 2.798
23.2 2.727
23.3 2.658
23.4 2.591
23.5 2.527
23.6 2.465
23.7 2.405
23.8 2.348
23.9 2.292
24.0 2.238
24.1 2.186
24.2 2.136
24.3 2.088
24.4 2.042
24.5 1.997
24.6 1.954
24.7 1.913
24.8 1.873
24.9 1.835
25.0 1.799
25.1 1.764
25.2 1.73
25.3 1.697
25.4 1.666
25.5 1.636
25.6 1.608
25.7 1.58
25.8 1.554
25.9 1.528
26.0 1.503
26.1 1.479
26.2 1.456
26.3 1.434
26.4 1.412
26.5 1.391
26.6 1.371
26.7 1.351
26.8 1.332
26.9 1.312
27.0 1.294
27.1 1.276
27.2 1.258
27.3 1.24
27.4 1.223
27.5 1.206
27.6 1.189
27.7 1.173
27.8 1.157
27.9 1.141
28.0 1.125
28.1 1.109
28.2 1.094
28.3 1.078
28.4 1.063
28.5 1.048
28.6 1.033
28.7 1.019
28.8 1.004
28.9 0.9898
29.0 0.9756
29.1 0.9616
29.2 0.9477
29.3 0.9339

14.7 7.068
14.8 7.387
14.9 7.726
15.0 8.081
15.1 8.45
15.2 8.83
15.3 9.215
15.4 9.601
15.5 9.979
15.6 10.34
15.7 10.69
15.8 11.0
15.9 11.27
16.0 11.5
16.1 11.68
16.2 11.81
16.3 11.88
16.4 11.9
16.5 11.88
16.6 11.81
16.7 11.71
16.8 11.59
16.9 11.45
17.0 11.31
17.1 11.17
17.2 11.03
17.3 10.9
17.4 10.79
17.5 10.7
17.6 10.64
17.7 10.59
17.8 10.56
17.9 10.56
18.0 10.57
18.1 10.61
18.2 10.65
18.3 10.71
18.4 10.77
18.5 10.84
18.6 10.9
18.7 10.95
18.8 10.99
18.9 11.01
19.0 11.01
19.1 10.99
19.2 10.93
19.3 10.85
19.4 10.74
19.5 10.61
19.6 10.45
19.7 10.26
19.8 10.05
19.9 9.831
20.0 9.593
20.1 9.344
20.2 9.086
20.3 8.823
20.4 8.557
20.5 8.289
20.6 8.023
20.7 7.758
20.8 7.497
20.9 7.241
21.0 6.99
21.1 6.745
21.2 6.507
21.3 6.275
21.4 6.052
21.5 5.836
21.6 5.628
21.7 5.427
21.8 5.235
21.9 5.05
22.0 4.874
22.1 4.705
22.2 4.543
22.3 4.389
22.4 4.241
22.5 4.101
22.6 3.966
22.7 3.838
22.8 3.716
22.9 3.6
23.0 3.488
23.1 3.382
23.2 3.281
23.3 3.184
23.4 3.091
23.5 3.003
23.6 2.918
23.7 2.837
23.8 2.76
2

9.1 0.7695
9.2 0.8085
9.3 0.8502
9.4 0.8949
9.5 0.943
9.6 0.9947
9.7 1.05
9.8 1.111
9.9 1.176
10.0 1.246
10.1 1.323
10.2 1.406
10.3 1.497
10.4 1.596
10.5 1.704
10.6 1.821
10.7 1.948
10.8 2.087
10.9 2.236
11.0 2.397
11.1 2.57
11.2 2.754
11.3 2.95
11.4 3.155
11.5 3.37
11.6 3.593
11.7 3.821
11.8 4.052
11.9 4.283
12.0 4.509
12.1 4.725
12.2 4.928
12.3 5.112
12.4 5.278
12.5 5.424
12.6 5.555
12.7 5.676
12.8 5.792
12.9 5.912
13.0 6.042
13.1 6.186
13.2 6.35
13.3 6.537
13.4 6.746
13.5 6.979
13.6 7.233
13.7 7.508
13.8 7.798
13.9 8.101
14.0 8.413
14.1 8.729
14.2 9.047
14.3 9.364
14.4 9.679
14.5 9.991
14.6 10.3
14.7 10.61
14.8 10.92
14.9 11.23
15.0 11.54
15.1 11.85
15.2 12.15
15.3 12.43
15.4 12.7
15.5 12.94
15.6 13.13
15.7 13.29
15.8 13.4
15.9 13.46
16.0 13.46
16.1 13.42
16.2 13.34
16.3 13.22
16.4 13.07
16.5 12.9
16.6 12.71
16.7 12.51
16.8 12.3
16.9 12.1
17.0 11.9
17.1 11.7
17.2 11.51
17.3 11.34
17.4 11.17
17.5 11.02
17.6 10.88
17.7 10.75
17.8 10.63
17.9 10.53
18.0 10.43
18.1 10.34
18.2 10.25
18.3 

28.4 1.106
28.5 1.093
28.6 1.081
28.7 1.069
28.8 1.058
28.9 1.046
29.0 1.035
29.1 1.024
29.2 1.014
29.3 1.003
29.4 0.9926
29.5 0.9823
29.6 0.9721
29.7 0.9621
29.8 0.9521
29.9 0.9423
30.0 0.9324
50 113
0.1 0.008559
0.2 0.00922
0.3 0.009933
0.4 0.0107
0.5 0.01152
0.6 0.0124
0.7 0.01333
0.8 0.01432
0.9 0.01536
1.0 0.01647
1.1 0.01763
1.2 0.01886
1.3 0.02015
1.4 0.02151
1.5 0.02293
1.6 0.02442
1.7 0.02599
1.8 0.02764
1.9 0.02936
2.0 0.03118
2.1 0.03308
2.2 0.03509
2.3 0.0372
2.4 0.03944
2.5 0.04183
2.6 0.04439
2.7 0.04717
2.8 0.05027
2.9 0.05383
3.0 0.05822
3.1 0.06425
3.2 0.07352
3.3 0.08482
3.4 0.09307
3.5 0.09926
3.6 0.1046
3.7 0.1097
3.8 0.1146
3.9 0.1195
4.0 0.1245
4.1 0.1296
4.2 0.1348
4.3 0.1402
4.4 0.1458
4.5 0.1515
4.6 0.1574
4.7 0.1636
4.8 0.1699
4.9 0.1766
5.0 0.1834
5.1 0.1905
5.2 0.1979
5.3 0.2056
5.4 0.2137
5.5 0.222
5.6 0.2307
5.7 0.2397
5.8 0.2492
5.9 0.259
6.0 0.2693
6.1 0.28
6.2 0.2913
6.3 0.303
6.4 0.3153
6.5 0.3281
6.6 0.3416
6.7 0.3557
6.8 0.3705
6.9 0.3861
7.0 0.4024


...

10.0 8.093
10.1 8.169
10.2 8.223
10.3 8.265
10.4 8.309
10.5 8.368
10.6 8.45
10.7 8.558
10.8 8.695
10.9 8.858
11.0 9.046
11.1 9.258
11.2 9.491
11.3 9.743
11.4 10.01
11.5 10.3
11.6 10.6
11.7 10.93
11.8 11.27
11.9 11.64
12.0 12.02
12.1 12.43
12.2 12.87
12.3 13.33
12.4 13.82
12.5 14.33
12.6 14.86
12.7 15.41
12.8 15.98
12.9 16.55
13.0 17.11
13.1 17.64
13.2 18.14
13.3 18.58
13.4 18.94
13.5 19.22
13.6 19.4
13.7 19.49
13.8 19.48
13.9 19.38
14.0 19.21
14.1 18.98
14.2 18.7
14.3 18.4
14.4 18.09
14.5 17.78
14.6 17.48
14.7 17.19
14.8 16.91
14.9 16.65
15.0 16.4
15.1 16.15
15.2 15.91
15.3 15.68
15.4 15.44
15.5 15.21
15.6 14.98
15.7 14.74
15.8 14.51
15.9 14.28
16.0 14.04
16.1 13.81
16.2 13.58
16.3 13.36
16.4 13.14
16.5 12.93
16.6 12.72
16.7 12.53
16.8 12.33
16.9 12.14
17.0 11.95
17.1 11.75
17.2 11.55
17.3 11.33
17.4 11.11
17.5 10.88
17.6 10.63
17.7 10.37
17.8 10.11
17.9 9.831
18.0 9.552
18.1 9.27
18.2 8.988
18.3 8.709
18.4 8.435
18.5 8.167
18.6 7.907
18.7 7.656
18.8 7.414
18.9 7.182
19.0 6.959
19.1 6.

1.7 0.07315
1.8 0.07843
1.9 0.08407
2.0 0.09012
2.1 0.0966
2.2 0.1036
2.3 0.1111
2.4 0.1192
2.5 0.128
2.6 0.1375
2.7 0.148
2.8 0.1595
2.9 0.1723
3.0 0.1865
3.1 0.2027
3.2 0.2213
3.3 0.2433
3.4 0.27
3.5 0.3043
3.6 0.3511
3.7 0.4203
3.8 0.5242
3.9 0.6438
4.0 0.7385
4.1 0.8128
4.2 0.88
4.3 0.9483
4.4 1.023
4.5 1.108
4.6 1.209
4.7 1.33
4.8 1.478
4.9 1.662
5.0 1.89
5.1 2.171
5.2 2.509
5.3 2.888
5.4 3.259
5.5 3.542
5.6 3.672
5.7 3.655
5.8 3.564
5.9 3.48
6.0 3.455
6.1 3.511
6.2 3.656
6.3 3.891
6.4 4.219
6.5 4.643
6.6 5.165
6.7 5.776
6.8 6.454
6.9 7.159
7.0 7.85
7.1 8.486
7.2 9.028
7.3 9.424
7.4 9.629
7.5 9.631
7.6 9.466
7.7 9.197
7.8 8.881
7.9 8.551
8.0 8.223
8.1 7.908
8.2 7.622
8.3 7.384
8.4 7.207
8.5 7.094
8.6 7.044
8.7 7.047
8.8 7.098
8.9 7.195
9.0 7.34
9.1 7.539
9.2 7.796
9.3 8.109
9.4 8.465
9.5 8.845
9.6 9.222
9.7 9.565
9.8 9.847
9.9 10.05
10.0 10.17
10.1 10.22
10.2 10.23
10.3 10.21
10.4 10.2
10.5 10.22
10.6 10.26
10.7 10.36
10.8 10.51
10.9 10.72
11.0 10.98
11.1 11.31
11.2 11.68
11.3 12.

28.9 1.445
29.0 1.427
29.1 1.409
29.2 1.391
29.3 1.374
29.4 1.357
29.5 1.34
29.6 1.323
29.7 1.306
29.8 1.29
29.9 1.274
30.0 1.257
50 157
0.1 0.02816
0.2 0.03036
0.3 0.03275
0.4 0.03536
0.5 0.03817
0.6 0.04122
0.7 0.04449
0.8 0.04802
0.9 0.0518
1.0 0.05587
1.1 0.06023
1.2 0.06491
1.3 0.06994
1.4 0.07533
1.5 0.08113
1.6 0.08738
1.7 0.0941
1.8 0.1014
1.9 0.1092
2.0 0.1177
2.1 0.127
2.2 0.1371
2.3 0.1482
2.4 0.1604
2.5 0.1739
2.6 0.189
2.7 0.2061
2.8 0.2255
2.9 0.2479
3.0 0.2744
3.1 0.3065
3.2 0.3471
3.3 0.4013
3.4 0.4799
3.5 0.6043
3.6 0.7909
3.7 0.9731
3.8 1.11
3.9 1.228
4.0 1.348
4.1 1.482
4.2 1.642
4.3 1.839
4.4 2.086
4.5 2.401
4.6 2.805
4.7 3.313
4.8 3.927
4.9 4.602
5.0 5.226
5.1 5.625
5.2 5.688
5.3 5.468
5.4 5.13
5.5 4.811
5.6 4.573
5.7 4.438
5.8 4.413
5.9 4.504
6.0 4.714
6.1 5.051
6.2 5.522
6.3 6.141
6.4 6.911
6.5 7.819
6.6 8.813
6.7 9.799
6.8 10.65
6.9 11.26
7.0 11.58
7.1 11.61
7.2 11.4
7.3 11.0
7.4 10.47
7.5 9.89
7.6 9.318
7.7 8.796
7.8 8.34
7.9 7.943
8.0 7.596
8.1 7.298
8.2 7.055

18.1 8.314
18.2 8.04
18.3 7.774
18.4 7.517
18.5 7.271
18.6 7.035
18.7 6.81
18.8 6.596
18.9 6.393
19.0 6.2
19.1 6.018
19.2 5.845
19.3 5.681
19.4 5.526
19.5 5.379
19.6 5.24
19.7 5.108
19.8 4.982
19.9 4.862
20.0 4.748
20.1 4.639
20.2 4.535
20.3 4.435
20.4 4.34
20.5 4.249
20.6 4.162
20.7 4.079
20.8 3.998
20.9 3.921
21.0 3.848
21.1 3.777
21.2 3.708
21.3 3.643
21.4 3.58
21.5 3.52
21.6 3.462
21.7 3.406
21.8 3.353
21.9 3.302
22.0 3.253
22.1 3.206
22.2 3.161
22.3 3.119
22.4 3.078
22.5 3.039
22.6 3.002
22.7 2.967
22.8 2.933
22.9 2.901
23.0 2.87
23.1 2.84
23.2 2.811
23.3 2.783
23.4 2.756
23.5 2.729
23.6 2.703
23.7 2.678
23.8 2.652
23.9 2.627
24.0 2.602
24.1 2.577
24.2 2.551
24.3 2.526
24.4 2.5
24.5 2.474
24.6 2.448
24.7 2.422
24.8 2.395
24.9 2.369
25.0 2.342
25.1 2.315
25.2 2.288
25.3 2.261
25.4 2.234
25.5 2.207
25.6 2.181
25.7 2.154
25.8 2.127
25.9 2.101
26.0 2.075
26.1 2.049
26.2 2.023
26.3 1.998
26.4 1.972
26.5 1.948
26.6 1.923
26.7 1.899
26.8 1.875
26.9 1.851
27.0 1.828
27.1 1.805
27.2 1.782


7.4 10.44
7.5 10.04
7.6 9.733
7.7 9.543
7.8 9.483
7.9 9.554
8.0 9.756
8.1 10.07
8.2 10.48
8.3 10.93
8.4 11.36
8.5 11.7
8.6 11.88
8.7 11.87
8.8 11.69
8.9 11.37
9.0 10.98
9.1 10.57
9.2 10.2
9.3 9.887
9.4 9.656
9.5 9.512
9.6 9.453
9.7 9.473
9.8 9.563
9.9 9.714
10.0 9.916
10.1 10.16
10.2 10.45
10.3 10.77
10.4 11.14
10.5 11.55
10.6 12.01
10.7 12.5
10.8 13.04
10.9 13.61
11.0 14.2
11.1 14.81
11.2 15.42
11.3 16.04
11.4 16.65
11.5 17.24
11.6 17.81
11.7 18.36
11.8 18.89
11.9 19.39
12.0 19.86
12.1 20.32
12.2 20.76
12.3 21.18
12.4 21.58
12.5 21.94
12.6 22.27
12.7 22.55
12.8 22.76
12.9 22.9
13.0 22.96
13.1 22.95
13.2 22.86
13.3 22.72
13.4 22.52
13.5 22.28
13.6 22.0
13.7 21.69
13.8 21.36
13.9 20.99
14.0 20.61
14.1 20.21
14.2 19.8
14.3 19.38
14.4 18.96
14.5 18.55
14.6 18.14
14.7 17.75
14.8 17.37
14.9 17.0
15.0 16.63
15.1 16.28
15.2 15.94
15.3 15.6
15.4 15.27
15.5 14.95
15.6 14.63
15.7 14.32
15.8 14.03
15.9 13.74
16.0 13.46
16.1 13.19
16.2 12.93
16.3 12.67
16.4 12.41
16.5 12.16
16.6 11.9
16.7 11.64
16

In [34]:
md_tst = {'50': {'100': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (0.1359,
                              0.14329,
                              0.151455,
                              0.160519)},
             '101': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (6.510E-03,
                              7.011E-03,
                              7.553E-03,
                              8.134E-03)}
            }
     }

In [27]:
def dict_to_df(ordered_nested_dict):
    reform = {(firstKey, secondKey, thirdKey): values for firstKey, middleDict in ordered_nested_dict.items() for secondKey, innerdict in middleDict.items() for thirdKey, values in innerdict.items()}
    df = pd.DataFrame(reform)
    df = df.T # transpose
    df.index.names=['Z', 'A', None]
    return df

In [28]:
# pprint.pprint(md)

In [29]:
df = dict_to_df(md)
df.head()

0         1         2         3         4         5    \
Z  A                                                                    
50 90 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006291  0.006776  0.007300  0.007862  0.008465  0.009107   
   91 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006350  0.006840  0.007369  0.007937  0.008545  0.009193   
   92 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   

                6        7        8        9     ...         290      291  \
Z  A                                             ...                        
50 90 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009790  0.01051  0.01128  0.01209   ...      0.9121   0.9004   
   91 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009882  0.01061  0.01138  0.01220   ...      0.9619   0.9492   
   92 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   

               292      293      294      295      296      297      298  \
Z  A                                                                       
50 90 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.8889   0.8775   0.8662   0.8550   0.8439   0.8329   0.8220   
   91 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.9365   0.9238   0.9111   0.8984   0.8858   0.8731   0.8605   
   92 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   

               299  
Z  A                
50 90 U    30.0000  
      fE1   0.8110  
   91 U    30.0000  
      fE1   0.8479  
   92 U    30.0000  

[5 rows x 300 columns]

In [30]:
df.loc[(50, 146, ['U', 'fE1']), :]

0        1        2        3        4        5       6    \
Z  A                                                                       
50 146 U    0.10000  0.20000  0.30000  0.40000  0.50000  0.60000  0.7000   
       fE1  0.01944  0.02095  0.02259  0.02436  0.02627  0.02831  0.0305   

                7        8        9     ...       290     291     292     293  \
Z  A                                    ...                                     
50 146 U    0.80000  0.90000  1.00000   ...    29.100  29.200  29.300  29.400   
       fE1  0.03284  0.03533  0.03799   ...     1.277   1.266   1.256   1.245   

               294     295     296     297    298     299  
Z  A                                                       
50 146 U    29.500  29.600  29.700  29.800  29.90  30.000  
       fE1   1.234   1.224   1.213   1.201   1.19   1.179  

[2 rows x 300 columns]

In [31]:
lorvec_df.loc[(50, 146, ['U', 'fE1']), :]

0        1         2         3         4         5         6    \
Z  A                                                                            
50 146 U    0.1000  0.20000  0.300000  0.400000  0.500000  0.600000  0.700000   
       fE1  0.1359  0.14329  0.151455  0.160519  0.170631  0.181975  0.194774   

                 7         8         9      ...            290        291  \
Z  A                                        ...                             
50 146 U    0.800000  0.900000  1.000000    ...      29.100000  29.200000   
       fE1  0.209311  0.225938  0.245101    ...       0.090461   0.093698   

                 292        293        294        295        296        297  \
Z  A                                                                          
50 146 U    29.30000  29.400000  29.500000  29.600000  29.700000  29.800000   
       fE1   0.09674   0.099111   0.100767   0.102081   0.103364   0.104466   

                  298        299  
Z  A                              
50 146 U    29.900000  30.000000  
       fE1   0.104862   0.104155  

[2 rows x 300 columns]

In [32]:
df.loc[(50, 146, ['U', 'fE1']), :] = lorvec_df.loc[(50, 146, ['U', 'fE1']), :]

In [33]:
df.loc[(50, 146, ['U', 'fE1']), :]

0        1         2         3         4         5         6    \
Z  A                                                                            
50 146 U    0.1000  0.20000  0.300000  0.400000  0.500000  0.600000  0.700000   
       fE1  0.1359  0.14329  0.151455  0.160519  0.170631  0.181975  0.194774   

                 7         8         9      ...            290        291  \
Z  A                                        ...                             
50 146 U    0.800000  0.900000  1.000000    ...      29.100000  29.200000   
       fE1  0.209311  0.225938  0.245101    ...       0.090461   0.093698   

                 292        293        294        295        296        297  \
Z  A                                                                          
50 146 U    29.30000  29.400000  29.500000  29.600000  29.700000  29.800000   
       fE1   0.09674   0.099111   0.100767   0.102081   0.103364   0.104466   

                  298        299  
Z  A                              
50 146 U    29.900000  30.000000  
       fE1   0.104862   0.104155  

[2 rows x 300 columns]

In [34]:
md_tst = {'50': {'100': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (0.1359,
                              0.14329,
                              0.151455,
                              0.160519)},
             '101': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (6.510E-03,
                              7.011E-03,
                              7.553E-03,
                              8.134E-03)}
            }
     }

In [35]:
df_tst = dict_to_df(md_tst)
df_tst

0         1         2         3
Z  A                                              
50 100 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.13590  0.143290  0.151455  0.160519
   101 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.00651  0.007011  0.007553  0.008134

In [36]:
df_tst = df_tst.T
pprint.pprint(df_tst.to_dict('list')) # dict is OK
df_tst = df_tst.T
print()
print(df_tst.index) # index is OK

{('50', '100', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '100', 'col2'): [0.1359, 0.14329, 0.151455, 0.160519],
 ('50', '101', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '101', 'col2'): [0.00651, 0.007011, 0.007553, 0.008134]}

MultiIndex(levels=[['50'], ['100', '101'], ['col1', 'col2']],
           labels=[[0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 1]],
           names=['Z', 'A', None])


In [37]:
table = df_tst.loc[('50', '100', ['col1', 'col2']), :]
table

0        1         2         3
Z  A                                            
50 100 col1  0.1000  0.20000  0.300000  0.400000
       col2  0.1359  0.14329  0.151455  0.160519

In [38]:
s = df_tst.loc[('50', '100', 'col1'), :]
s

0    0.1
1    0.2
2    0.3
3    0.4
Name: (50, 100, col1), dtype: float64

In [39]:
stocks

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [40]:
stocks.loc[('AAPL', '2016-10-03'), :] # rows, columns
stocks.loc[('AAPL', '2016-10-03'), 'Close']
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), 'Close']
stocks.loc[('AAPL', ['2016-10-03', '2016-10-04']), 'Close']
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']), :]

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900

In [41]:
md = {'50': {'100': {'col1': ('0.100',
                              '0.200',
                              '0.300',
                              '0.400'),
                     'col2': ('6.263E-03',
                              '6.746E-03',
                              '7.266E-03',
                              '7.825E-03')},
             '101': {'col1': ('0.100',
                              '0.200',
                              '0.300',
                              '0.400'),
                     'col2': ('6.510E-03',
                              '7.011E-03',
                              '7.533E-03',
                              '8.134E-03')}
            }
     }

In [42]:
reform = {(firstKey, secondKey, thirdKey): values for firstKey, middleDict in md.items() for secondKey, innerdict in middleDict.items() for thirdKey, values in innerdict.items()}

In [43]:
df = pd.DataFrame(reform)

In [44]:
df

50                             
     100               101           
    col1       col2   col1       col2
0  0.100  6.263E-03  0.100  6.510E-03
1  0.200  6.746E-03  0.200  7.011E-03
2  0.300  7.266E-03  0.300  7.533E-03
3  0.400  7.825E-03  0.400  8.134E-03

In [45]:
md2={}
for i in df.columns:
    if i[0] not in md2.keys():
        md2[i[0]]={}
    if i[1] not in md2[i[0]].keys():
        md2[i[0]][i[1]]={}
    md2[i[0]][i[1]][i[2]]=tuple(df[i[0]][i[1]][i[2]].values)

In [46]:
pprint.pprint(md2)

{'50': {'100': {'col1': ('0.100', '0.200', '0.300', '0.400'),
                'col2': ('6.263E-03', '6.746E-03', '7.266E-03', '7.825E-03')},
        '101': {'col1': ('0.100', '0.200', '0.300', '0.400'),
                'col2': ('6.510E-03', '7.011E-03', '7.533E-03', '8.134E-03')}}}


In [47]:
md2 == md

True